In [1]:
import pandas as pd

def get_date(filename):
    result = []
    for df in pd.read_csv(filename, chunksize=1000000):
        result.append(df)
    result = pd.concat(result,ignore_index=True,axis=0)
    return result

In [2]:
#获取全量数据
train=get_date('./security_train.csv')
train.head(5)

file_id  label                     api   tid  index
0        1      5              LdrLoadDll  2488      0
1        1      5  LdrGetProcedureAddress  2488      1
2        1      5  LdrGetProcedureAddress  2488      2
3        1      5  LdrGetProcedureAddress  2488      3
4        1      5  LdrGetProcedureAddress  2488      4

In [3]:
test=get_date('./security_test.csv')
test

file_id                 api   tid  index
0               1       RegOpenKeyExA  2332      0
1               1           CopyFileA  2332      1
2               1      OpenSCManagerA  2332      2
3               1      CreateServiceA  2332      3
4               1       RegOpenKeyExA  2468      0
...           ...                 ...   ...    ...
79288370    12955        Thread32Next  2740   1446
79288371    12955        Thread32Next  2740   1447
79288372    12955             NtClose  2740   1448
79288373    12955       __exception__  2740   1449
79288374    12955  NtTerminateProcess  2740   1450

[79288375 rows x 4 columns]

In [4]:
import psutil
mem = psutil.virtual_memory()
print('总内存',mem.total/1024/1024)
print('已使用内存',mem.used/1024/1024)
print('剩余内存',mem.free/1024/1024)
print('内存使用率',mem.percent)

总内存 16110.78515625
已使用内存 11865.734375
剩余内存 4245.05078125
内存使用率 73.7


In [5]:
#文件以pkl格式保存
import pickle
with open('train.pkl','wb') as f:
    pickle.dump(train,f)

with open('test.pkl','wb') as f:
    pickle.dump(test,f)

In [6]:
%%time
#读取pkl文件
import pickle
with open('train.pkl','rb') as f:
    train = pickle.load(f)

with open('test.pkl','rb') as f:
    test = pickle.load(f)

CPU times: total: 10.8 s
Wall time: 12.1 s


In [7]:
#数据探索
#13887个文件
train['file_id'].value_counts()
#api特征
train['api'].describe()

count                   89806693
unique                       295
top       LdrGetProcedureAddress
freq                    10704305
Name: api, dtype: object

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#将训练集和测试集的api字段进行合并，以免测试集中出现训练集没有的字段
df_all=pd.concat([train,test])
df_all['api']=le.fit_transform(df_all['api'])
df_all['api']
#提取train['api']
train['api'] = df_all[df_all['label'].notnull()]['api']
test['api'] = df_all[df_all['label'].isnull()]['api']
train

file_id  label  api   tid  index
0               1      5  135  2488      0
1               1      5  134  2488      1
2               1      5  134  2488      2
3               1      5  134  2488      3
4               1      5  134  2488      4
...           ...    ...  ...   ...    ...
89806688    13887      2  152  2336    618
89806689    13887      2  152  2336    619
89806690    13887      2  152  2336    620
89806691    13887      2  152  2336    621
89806692    13887      2  197  2336    622

[89806693 rows x 5 columns]

In [9]:
#清理不需要的变量
import gc
del df_all
gc.collect()

0

In [11]:
#提取特征
#根据file_id进行分组，进行聚合统计
def get_features(df):
    df_file=df.groupby('file_id')
    #dfl作为最终结果
    if 'label' in df.columns:
        dfl=df.drop_duplicates(subset=['file_id','label'],keep='first')
    else:
        dfl=df.drop_duplicates(subset=['file_id'],keep='first')
    dfl=dfl.sort_values(by='file_id')
    #提取多个特征的统计特征 api、tid、index
    featurtes=['api','tid','index']
    for f in featurtes:
        dfl[f+'_count']=df_file[f].count().values
        dfl[f+'_nunique']=df_file[f].nunique().values
        dfl[f+'_max']=df_file[f].max().values
        dfl[f+'_min']=df_file[f].min().values
        dfl[f+'_std']=df_file[f].std().values
        dfl[f+'_mean']=df_file[f].mean().values
        dfl[f+'_median']=df_file[f].median().values
        dfl[f+'_ptp']=dfl[f+'_max']-dfl[f+'_min']
    return dfl

In [12]:
df_train=get_features(train)
df_train.head(5)
df_test=get_features(test)
df_test.head(5)

file_id  api   tid  index  api_count  api_nunique  api_max  api_min  \
0           1  226  2332      0         97           15      262       13   
97          2  226  2472      0       1361           40      261        6   
1458        3   95  2344      0         16            9      257       16   
1474        4  135  2452      0        193           34      262       13   
1667        5   95  2332      0        803           34      261       16   

        api_std    api_mean  ...  tid_median  tid_ptp  index_count  \
0     50.338954  155.989691  ...      2544.0      236           97   
97    49.557795  138.025716  ...      2524.0      276         1361   
1458  75.152179  111.375000  ...      2344.0        0           16   
1474  45.377632  172.217617  ...      2452.0      132          193   
1667  55.316733  168.490660  ...      2376.0      448          803   

      index_nunique  index_max  index_min   index_std  index_mean  \
0                31         30          0    9.210466   14.443299   
97              681        680          0  196.515744  339.750184   
1458             16         15          0    4.760952    7.500000   
1474            193        192          0   55.858452   96.000000   
1667            268        267          0   77.317048  133.333748   

      index_median  index_ptp  
0             14.0         30  
97           340.0        680  
1458           7.5         15  
1474          96.0        192  
1667         133.0        267  

[5 rows x 28 columns]

In [13]:
%%time
df_train.to_pickle('./df_train.pkl')
df_test.to_pickle('./df_test.pkl')
with open('df_train.pkl','rb') as f:
    df_train = pickle.load(f)
with open('df_test.pkl','rb') as f:
    df_test = pickle.load(f)

CPU times: total: 15.6 ms
Wall time: 12 ms


In [14]:
%%time
import lightgbm as lgb
clf = lgb.LGBMClassifier(
           num_leaves=2**5-1, reg_alpha=0.25, reg_lambda=0.25, objective='multiclass',
            max_depth=-1, learning_rate=0.005, min_child_samples=3, random_state=2024,
            n_estimators=2000, subsample=1, colsample_bytree=1)
clf.fit(df_train.drop(['file_id','label'],axis=1),df_train['label'])

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000683 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5667
[LightGBM] [Info] Number of data points in the train set: 13887, number of used features: 26
[LightGBM] [Info] Start training from score -1.025925
[LightGBM] [Info] Start training from score -3.320108
[LightGBM] [Info] Start training from score -2.451970
[LightGBM] [Info] Start training from score -2.829404
[LightGBM] [Info] Start training from score -4.933538
[LightGBM] [Info] Start training from score -1.174900
[LightGBM] [Info] Start training from score -3.294542
[LightGBM] [Info] Start training from score -2.234192
CPU times: total: 3min 4s
Wall time: 24.6 s


LGBMClassifier(colsample_bytree=1, learning_rate=0.005, min_child_samples=3,
               n_estimators=2000, objective='multiclass', random_state=2024,
               reg_alpha=0.25, reg_lambda=0.25, subsample=1)

In [25]:
%%time
result=clf.predict_proba(df_test.drop(['file_id'],axis=1))
result = pd.DataFrame(result,columns=['prob0','prob1','prob2','prob3','prob4','prob5','prob6','prob7'])
result['file_id']=df_test['file_id'].values
columns=['file_id','prob0','prob1','prob2','prob3','prob4','prob5','prob6','prob7']
result.to_csv('./baseline1.csv',columns=columns,index=False)

CPU times: total: 30.9 s
Wall time: 3.94 s
